# 知识工程-作业9 基于知识图谱的问答系统
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image.png)


很好，这次代码格式化没有报错。

Pylance 似乎也没有明显问题。

## 实验环境准备

采用上次的作业专属环境，为了跑通最新方法，使用3.12 和 torch 2.6

```bash
conda create -n assignments python=3.12
conda activate assignments
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

本次作业似乎没有新的依赖，只是用到了 transformers

```python
import json
import numpy as np
from transformers import BertTokenizer, BertModel
```



## 原理回顾和课件复习



课上详细介绍了 Knowledge-based Question Answering

首先区分了一下属性和关系，属性是 实体, 属性类型, 字符串； 关系是 实体，关系类型，实体。



## 1.1 基于知识图谱的问答系统

根据助教老师的要求，我们有四步要做

1. 使用 Python 解析 zhishime.json 文件，创建知识图谱
2. 实现头实体检索模块（使用正向最大匹配或命名实体识别）
3. 使用预训练模型计算问题与关系的相似度
4. 提取答案并评估准确性


### 1. 使用 python 解析 zhishime.json 文件，并将解析出的 dict 保存为文件

注意到 zhishime 实际上是一个 jsonl文件，一行是一个json，每一行是

```json
{
    "_id": {"$oid": "5a4a0579b63209a91d0c41c7"},
    "head": "1987大悬案",
    "relation": "监制",
    "tail": "狄诺迪洛伦提斯\u003cbr/\u003eRichardRoth\u003cbr/\u003e柏尼·威廉斯",
}
```

这样的格式, 也就是说有 _id, head, relation 和 tail 

这是知识图谱的典型的三元组。

老师已经给了我们 preprocess.py 的初步实现。

```python
kg = {
        "head2id": head2id,
        "tail2id": tail2id,
        "relation2id": relation2id,
        "relation_triplets": relation_triplets,
    }
```

这个函数的目标是为了得到 head tail 的id，其实和没处理差不多。


In [ ]:
import json
import os
from tqdm import tqdm

def preprocess():
    """预处理知识图谱数据，构建实体映射和三元组索引"""
    # 初始化数据结构
    kg = {
        "head2id": {},     # 头实体到ID的映射
        "tail2id": {},     # 尾实体到ID的映射  
        "relation2id": {}, # 关系到ID的映射
        "relation_triplets": []  # 存储(hid, rid, tid)形式的三元组
    }

    try:
        # 确保输出目录存在
        os.makedirs("./processed", exist_ok=True)
        
        # 读取原始JSONL文件（注意：使用..表示上级目录）
        with open("../zhishime.json", "r", encoding="utf-8") as f:
            # 逐行解析JSON对象（适用于JSON Lines格式）
            raw_relation_data = [json.loads(line) for line in f]
            
            # 等价写法（更易理解）：
            # raw_relation_data = []
            # for line in f:
            #     data = json.loads(line)
            #     raw_relation_data.append(data)

        # 遍历每个三元组进行索引构建
        bar = tqdm(raw_relation_data)
        for item in bar:
            head = item["head"]
            relation = item["relation"]
            
            # 处理包含换行符的尾实体（示例数据中的<br/>分隔符）
            tail = item["tail"].replace("\u003cbr/\u003e", ", ")  # 将HTML换行符转换为逗号分隔
            
            # 构建实体ID映射（自动递增分配ID）
            # head2id.setdefault等效写法，但更推荐当前写法
            if head not in kg["head2id"]:
                kg["head2id"][head] = len(kg["head2id"])
                
            if tail not in kg["tail2id"]:
                kg["tail2id"][tail] = len(kg["tail2id"])
                
            if relation not in kg["relation2id"]:
                kg["relation2id"][relation] = len(kg["relation2id"])
            
            # 构建三元组索引
            hid = kg["head2id"][head]
            rid = kg["relation2id"][relation]
            tid = kg["tail2id"][tail]
            kg["relation_triplets"].append((hid, rid, tid))

        # 打印统计信息
        print(f"[统计] 头实体数: {len(kg['head2id'])} | 尾实体数: {len(kg['tail2id'])} | 关系类型数: {len(kg['relation2id'])}")
        print(f"[统计] 总三元组数: {len(kg['relation_triplets'])}")

        # 保存处理结果
        with open("./processed/kg.json", "w", encoding="utf-8") as json_file:
            json.dump(kg, json_file, 
                     ensure_ascii=False,  # 保留非ASCII字符原文
                     indent=4)           # 美化格式便于查看
            
    except FileNotFoundError:
        print("错误：未找到原始数据文件，请检查路径是否正确")
    except json.JSONDecodeError as e:
        print(f"JSON解析错误：第{e.lineno}行数据格式异常，错误详情：{e.msg}")

if __name__ == "__main__":
    preprocess()


我们稍微重构了下，让整个功能更加稳定。

![alt text](image-2.png)

### 2. 实现头实体检索模块（使用正向最大匹配或命名实体识别）

在实现这个函数之前，我们需要明确一个问题，
- what：到底什么是头实体？
- 在一个question里面，头实体应该是一个还是多个？

事实上，这个和对问题的模板预设有关，这次实验我们假设问题类似于 “{HEAD} 的 {RELATION}?” ， 回答是 “{TAIL}”， 所以认为问题中就只有一个实体。例如："周杰伦的出生日期是什么？" → 头实体是"周杰伦"。

实际上问题中可能包含多个实体，但通常只有一个是头实体

例如："周杰伦和林俊杰谁的粉丝多？" → 这种情况复杂一些，可能需要查询多个头实体。


刚才我们定义了 "head2id": {},     # 头实体到ID的映射

所以，我们只需要做一个“多字符串(模式)匹配", 找到 question 字符串中的那一个实体就好。

所谓 正向最大匹配法，就是在问题字符串中，尝试匹配知识图谱中最长的实体名称，优先匹配最长的实体名称，因为这样更可能是完整的实体。 那么相应的逻辑就很简单了


In [ ]:
def search_head_entity(kg: dict, question: str) -> str:
    """基于正向最大匹配的头实体识别
    Args:
        kg: 知识图谱字典，包含head2id等字段
        question: 待查询的问题文本
    Returns:
        匹配成功的头实体字符串，未找到返回None
    """
    # 获取所有可能的头实体
    all_heads = list(kg['head2id'].keys())
    
    # 对头实体按长度排序，优先匹配长的实体
    all_heads.sort(key=len, reverse=True)
    
    # 遍历所有头实体，检查是否在问题中出现
    for head in all_heads:
        if head in question:
            return head
    
    # 如果没有找到匹配的头实体
    return ""

为了提高处理速度，我们使用缓存

In [ ]:
sorted_heads = None

def search_head_entity(kg: dict, question: str) -> str:
    """基于正向最大匹配的头实体识别
    Args:
        kg: 知识图谱字典，包含head2id等字段
        question: 待查询的问题文本
    Returns:
        匹配成功的头实体字符串，未找到返回None
    """
    # 获取所有可能的头实体
    global sorted_heads
    if sorted_heads is None:
        all_heads = list(kg['head2id'].keys())
        
        # 对头实体按长度排序，优先匹配长的实体
        sorted_heads = sorted(all_heads, key=len, reverse=True)
    
    # 遍历所有头实体，检查是否在问题中出现
    for head in sorted_heads:
        if head in question:
            return head
    
    # 如果没有找到匹配的头实体
    return ""

注意到后面的代码是这样的

head = search_head_entity(kg, question)
        if head is None:

为了不让类型有问题（不想用optional），我觉得应该改成
if head=="":

### 3. 使用预训练模型计算问题与关系的相似度

检查 main.py 代码，发现原本代码这里写错了

relations = list(kg[head].keys())

我们在kg中根本没有存储从 head 映射到 关系本身的信息！

所以我们需要重新修改 preprocess.py 把这个信息假如进去才对。

刚才处理json的时候搞了半天id其实是无用功，根本不需要id，核心问题是 一个head有多少个相关的relation。



同理，这段代码也不对

answer = kg[head][max_relation]

这里要解决的核心问题是，给定一个head和一个relation，找到对应的tail的集合。

我们首先加上代码

```python
# 构建头实体到关系的映射
if head not in kg["head2relations"]:
    kg["head2relations"][head] = set()
kg["head2relations"][head].add(relation)

# 构建头实体到关系和答案的映射
if (head, relation) not in kg["head_relations2answers"]:
    kg["head_relations2answers"][(head, relation)] = ""
kg["head_relations2answers"][(head, relation)] += f"{tail}, "
```

重新运行 python preprocess.py

现在可以看到


In [1]:
import json
with open("data/processed/kg.json", "r", encoding="utf-8") as f:
    kg = json.load(f)
kg.keys()

dict_keys(['head2id', 'tail2id', 'relation2id', 'relation_triplets'])

在main中使用 
```python
        relations = list(kg["head2relations"][head])
        ...
        answer = kg["head_relations2answers"][(head, max_relation)]
```

